# simple example

In [1]:
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)
warnings.simplefilter(action = 'ignore', category = NumbaDeprecationWarning)
warnings.simplefilter(action = 'ignore', category = NumbaPendingDeprecationWarning)
import sys
sys.path.append('../')
from gwas_class_auto import *
dd = pd.read_csv(f'data_dict_example.csv')
df = pd.read_csv(f'raw_data.csv', dtype = {'rfid': str}).drop_duplicates(subset = 'rfid')

traits_ = []
traits_d = []
gwas = gwas_pipe(path = f'',
             project_name = 'test',
             raw = df,
             traits = traits_ ,
             trait_descriptions = traits_d,
             all_genotypes = '/projects/ps-palmer/tsanches/gwaspipeline/gwas/zzplink_genotypes/round10', #'round9_1.vcf.gz',
             founder_genotypes = '/projects/ps-palmer/hs_rats/Ref_panel_mRatBN7.2/Ref_panel_mRatBN7_2_chr_GT',
             snpeff_path ='snpEff/',
             phewas_db = '/projects/ps-palmer/tsanches/gwaspipeline/gwas/phewasdb.parquet.gz',
             threads = 8)

TypeError: gwas_pipe.__init__() got an unexpected keyword argument 'data'

In [ ]:
gwas.regressout(data_dictionary = dd, covariates_threshold=0.02, verbose = True)
gwas.subsetSamplesFromAllGenotypes(sourceFormat = 'plink')
gwas.generateGRM()
gwas.snpHeritability()
gwas.BLUP()
gwas.GWAS()
gwas.addGWASresultsToDb(researcher='', round_version='10.0.0', gwas_version='0.1.1')
qtls = gwas.callQTLs( NonStrictSearchDir=False)
gwas.annotate(qtls)
gwas.locuszoom(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv')) 
gwas.effectsize(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv')) 
gwas.genetic_correlation_matrix()
gwas.manhattanplot(display = False)
gwas.porcupineplot(pd.read_csv(f'{gwas.path}/results/qtls/finalqtl.csv'), display = False)
gwas.phewas(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv').set_index('SNP'), annotate=True, pval_threshold = 1e-4, nreturn = 1, r2_threshold = .4) 
gwas.eQTL(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv').set_index('SNP'), annotate= True)
gwas.sQTL(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv').set_index('SNP'))
gwas.report(round_version='10.0.0') ### add gwas version
gwas.store(researcher='',round_version='10.0.0', gwas_version='0.1.1',  remove_folders=False)
gwas.copy_results()

# how to regressout per group

In [ ]:
def regress(q, t):
    gwas = gwas_pipe(path = f'',
             project_name = 'u01_peter_kalivas_clusters_june2023',
             data = q,
             traits = t ,
             trait_descriptions= t,
             all_genotypes = '/projects/ps-palmer/tsanches/gwaspipeline/gwas/zzplink_genotypes/round10', #'round9_1.vcf.gz',
             snpeff_path='../snpEff/',
             threads = 8)
    return gwas.regressout(data_dictionary = f'data_dict_u01_peter_kalivas_clusters.csv', 
               covariates_threshold=0.02, verbose = False)
    

In [ ]:
traits_interest = ['escalation_of_heroin_intake_12h', 'total_heroin_consumption', 'bp', 'tail_flick_bl2'] 
regressed_out = dff.groupby('site').apply(lambda x: regress(x,traits_interest))

In [ ]:
regressed_out = regressed_out.reset_index(drop = True)

In [ ]:
traits = 'regressedlr_'+pd.Series(traits_interest) 
regressed_out[traits] = statsReport.quantileTrasformEdited(regressed_out,traits )

In [ ]:
traits_d = get_trait_descriptions_f(dd, traits_)
gwas = gwas_pipe(path = f'',
             all_genotypes = '/projects/ps-palmer/tsanches/gwaspipeline/gwas/zzplink_genotypes/round10', #'round9_1.vcf.gz',
             data = regressed_out,
             snpeff_path='../snpEff/',
             project_name = 'u01_peter_kalivas_clusters_june2023',
             traits = traits ,
             trait_descriptions= traits_d,
             threads = 8)

In [ ]:
gwas.subsetSamplesFromAllGenotypes(sourceFormat = 'plink')
gwas.generateGRM()
gwas.snpHeritability()
gwas.BLUP()
gwas.GWAS()
gwas.addGWASresultsToDb(researcher='', round_version='10.0.0', gwas_version='0.1.1')
qtls = gwas.callQTLs( NonStrictSearchDir=False)
gwas.annotate(qtls)
gwas.locuszoom(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv')) 
gwas.effectsize(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv')) 
gwas.genetic_correlation_matrix()
gwas.manhattanplot(display = False)
gwas.porcupineplot(pd.read_csv(f'{gwas.path}/results/qtls/finalqtl.csv'), display = False)
gwas.phewas(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv').set_index('SNP'), annotate=True, pval_threshold = 1e-4, nreturn = 1, r2_threshold = .4) 
gwas.eQTL(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv').set_index('SNP'), annotate= True)
gwas.sQTL(pd.read_csv(f'{gwas.path}results/qtls/finalqtl.csv').set_index('SNP'))
gwas.report( round_version='10.0.0') ### add gwas version
gwas.store(researcher='',round_version='10.0.0', gwas_version='0.1.1',  remove_folders=False)
gwas.copy_results()